# Simple Decoder, Version 3

From Karpathy's [Let's build GPT: from scratch, in code, spelled out.](https://www.youtube.com/watch?v=kCc8FmEb1nY&t=6096s), at time 1:00:19.

Here, we implement scaled dot-product attention as per Attention Is All You Need, page 4, Scaled Dot-Product Attention.

#### NOTES

1. implemented scaled dot-product attention
1. implemented a single attention head
1. lowering `learning_rate` as _self-attention (attention in general?) can't tolerate very high learning rates_
1. incxreaing `max_iters` and `eval_interval` to compensate for lowered `learning_rate`

In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F

In [2]:
# hyperparameters
batch_size = 32   # how many independent sequences will we process in parallel?
block_size = 8    # what is the maximum context length for prediction?
max_iters = 5000
eval_interval = 500
learning_rate = 1e-3
device = "cuda" if torch.cuda.is_available() else "cpu"
eval_iters = 200
n_embed = 32
# --------------


with open("input.txt", "r", encoding="UTF-8") as fin:
    text = fin.read()

# vocabulary of individual chars seen in text
chars = sorted(list(set(text)))
vocab_size = len(chars)

# character-level tokenization for the character-based transformer
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s]           # encoder: take a string, output a list of int
decode = lambda l: ''.join([itos[i] for i in l])  # decoder: take a list of int, output a string

# train and test splits, 90:10
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data))
train_data = data[:n]
valid_data = data[n:]


def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == "train" else valid_data
    
    ix = torch.randint(len(data) - block_size, (batch_size,))
    
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    
    x, y = x.to(device), y.to(device)
    
    return x, y


@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ["train", "valid"]:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X,Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [3]:
%%time

# for reproducibility
torch.manual_seed(1337)


class Head(nn.Module):
    """ A single head for self-attention """
    
    def __init__(self, head_size):
        super().__init__()
        self.query = nn.Linear(n_embed, head_size, bias=False)
        self.key = nn.Linear(n_embed, head_size, bias=False)
        self.value = nn.Linear(n_embed, head_size, bias=False)
        self.register_buffer("tril", torch.tril(torch.ones(block_size, block_size)))

    def forward(self, x):
        B,T,C = x.shape
        q = self.query(x)   # (B,T,C)
        k = self.key(x)     # (B,T,C)
        
        # compute attention scores; the "affinities"
        wei = q @ k.transpose(-2,-1) * C**-0.5
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float("-inf"))
        wei = F.softmax(wei, dim=-1)
        
        v = self.value(x)
        out = wei @ v
        return out


class SimpleDecoder(nn.Module):
    """ A simple Decoder implementation that we will slowly implement """
    
    def __init__(self):
        super().__init__()
        
        # each token directly reads off the logits
        # for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embed)
        self.position_embedding_table = nn.Embedding(block_size, n_embed)
        self.sa_head = Head(n_embed)
        self.lm_head = nn.Linear(n_embed, vocab_size)
 
    def forward(self, idx, targets=None):
        B, T = idx.shape
        
        # idx and targets are both (B,T) tensor of ints
        token_embeddings = self.token_embedding_table(idx)    # (B,T,C)
        position_embeddings = self.position_embedding_table(torch.arange(T, device=device)) #  (T,C)
        x = token_embeddings + position_embeddings            # (B,T,C)
        x = self.sa_head(x)                                   # apply a single head of attention
        logits = self.lm_head(x)                              # (B,T,vocab_size)
        
        
        if targets is None:
            loss = None
        else:
            B,T,C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        # idx is (B,T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            
            # get predictions
            logits, loss = self(idx_cond)
            
            # focus only on the last time step (bigram)
            logits = logits[:, -1, :] # becomes (B, c)
            
            # apply softmax to get probabilitis
            probs = F.softmax(logits, dim=-1) # (B, C)
            
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx


model = SimpleDecoder()
m = model.to(device)

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=learning_rate)

for iter in range(max_iters):
    
    # every once in a while, evaluate the loss on train and validation sets
    if iter % eval_interval == 0:
        losses = estimate_loss()
        print(f"step {iter}: train loss is {losses['train']:.4f}, validation loss {losses['valid']:.4f}")
    
    # sample a batch of data
    xb, yb = get_batch("train")
    
    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# generate from the model
context = torch.zeros((1,1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=500)[0].tolist()))
print("==========================")
print()

step 0: train loss is 4.1990, validation loss 4.2031
step 500: train loss is 2.6944, validation loss 2.7091
step 1000: train loss is 2.5214, validation loss 2.5316
step 1500: train loss is 2.4762, validation loss 2.4833
step 2000: train loss is 2.4408, validation loss 2.4517
step 2500: train loss is 2.4279, validation loss 2.4433
step 3000: train loss is 2.4126, validation loss 2.4314
step 3500: train loss is 2.3950, validation loss 2.4190
step 4000: train loss is 2.4033, validation loss 2.3961
step 4500: train loss is 2.3976, validation loss 2.4041

Whent iknt,
Thowi, ht son, bth

Hiset bobe ale.
S:
O-' st dalilanss: rant he us he, vet?
Wed las ate awice my.

HDET:
ANGo oug
Yowns, tof isth bot mil ndill, aes iree sen cie lat Herid ovets, and Win ngarigoerabous lelind peal.
-hule onchiry ptugr aiss hew ye wllinde norod atelaves
Momy yowod mothake ont-wou whrt eiiby we ati fourive wee, ired thoouso er; thu the banterupt f so;
ARID Wam:
ENGCI inleront ffaf Pre?

Whiom.

He-
LIERCKENIGUIC